The third functionality of the privaSEE app is to recommend users services that score well in their particular domains of interest. Users may ask questions like "what's a service similar to X app but has a higher score"? Or, they can ask "if I'm looking for an app to do such and such, which app should I use?".

In this script, we assume that we have a database of scored apps. We have service names and scores. We need to find a way to categorize apps based on more granular categories than just "social media" or "music". Ie we must take into account the size, functionality, etc. Then, we would layer an LLM on top of this so that users can converse.

In [101]:
import pandas as pd
import os
from google_play_scraper import app
from google_play_scraper import search
from fuzzywuzzy import fuzz
import re

In [12]:
path = '/Users/saronson/Documents/03_Classes/2024/02_Advanced_Practical_Data_Science/05_Project/ac215_PrivaSEE'
os.chdir(path)
print(os.getcwd()) #make sure wd is set to the project folder from github

/Users/saronson/Documents/03_Classes/2024/02_Advanced_Practical_Data_Science/05_Project/ac215_PrivaSEE


In [72]:
#for now, we'll use an old version of the dataset. This comes from the intermediate data file which includes service names and ratings.
#This is just for a proof of concept - we'd replace that df with the scores that our model gives

df = pd.read_csv('intermediate_data_files/services_and_ratings.csv')
#drop the "Unnamed: 0" column
df = df.drop(columns=['Unnamed: 0'])
#drop rows where Rating is missing, bc we are assuming all services will have a rating bc they will have gone through our model
df = df.dropna(subset=['Rating'])
df = df[df['Rating'] != "No rating yet"]
df = df.reset_index(drop=True)
df

,Service,Rating
0,AliExpress,D
1,MySudo,B
2,Rec Room,D
3,ISODME,D
4,No Kid Hungry,D
...,...,...
1792,Next of Windows,C
1793,Replit,C
1794,Girls Who Code,D
1795,Revolut,D


In [24]:
#find granular data on each service
result = app(
    'com.nianticlabs.pokemongo',
    lang='en', # defaults to 'en'
    country='us' # defaults to 'us'
)
print(result)

{'title': 'Pokémon GO', 'description': 'New! Now you can battle other Pokémon GO Trainers online! Try the GO Battle League today!\r\n\r\nJoin Trainers across the globe who are discovering Pokémon as they explore the world around them. Pokémon GO is the global gaming sensation that has been downloaded over 1 billion times and named “Best Mobile Game” by the Game Developers Choice Awards and “Best App of the Year” by TechCrunch.\r\n_______________\r\n\r\nUncover the world of Pokémon: Explore and discover Pokémon wherever you are!\r\n \r\nCatch more Pokémon to complete your Pokédex!\r\n \r\nJourney alongside your Buddy Pokémon to help make your Pokémon stronger and earn rewards!\r\n\r\nCompete in epic Gym battles and...Team up with other Trainers to catch powerful Pokémon during Raid Battles!\r\n \r\nIt’s time to get moving—your real-life adventures await! Let’s GO!\r\n_______________\r\n\r\nNotes: \r\n- This app is free-to-play and offers in-game purchases. It is optimized for smartphone

In [36]:
result = search(
    "facebook",
    lang="en",  # defaults to 'en'
    country="us",  # defaults to 'us'
    n_hits=3  # defaults to 30 (= Google's maximum)
)
print(result)

[{'appId': 'com.facebook.katana', 'icon': 'https://play-lh.googleusercontent.com/KCMTYuiTrKom4Vyf0G4foetVOwhKWzNbHWumV73IXexAIy5TTgZipL52WTt8ICL-oIo', 'screenshots': ['https://play-lh.googleusercontent.com/yltemK6xddx3AdaqfoI2RYFB1rHDpDxkZZl8I_3uWqOR_sGAj20sCE0a1hS_zL59mWs', 'https://play-lh.googleusercontent.com/uPC3VsBbDU5IakuOsLLFvFppuKyV6IDEIrX8EZRlR4EiRisRuSNrYCxhc30HiduWdkVi', 'https://play-lh.googleusercontent.com/LqFs9L14viQoszsQtSqqDVk3m4bKXlDSYUIYii0e7U_y0oXxASPtoxj2Pt4gV8KHDDU', 'https://play-lh.googleusercontent.com/DH_oPwYKAm6_3xjvfG8yio4xQayew62B658_4w3nIyxGqRiQncG8jX3fa3qmRZ_EQxU', 'https://play-lh.googleusercontent.com/3aTPofuVkgP4XlICCyVPZgp8ZGDXPeezeZgNh4YtCY6VUNqCg06AXYGPWRvAq-jF0fk', 'https://play-lh.googleusercontent.com/ioi2RTrBkBtTvlXtksF9oOmmEDaOswgm02oIg6KRh5QsmdXBTUxCe7zeOCglCeib_Lxy', 'https://play-lh.googleusercontent.com/b2kukObx5S3NrTPDmSEz3_GtiZUkhhXErUQgE4gBxbvYKVyJH3M19ADpkCO4UHztVg', 'https://play-lh.googleusercontent.com/WGvXMI3Vshmv6IMws7YEkzXUrV1vOh

In [74]:
df = df.head(20)
df

,Service,Rating
0,AliExpress,D
1,MySudo,B
2,Rec Room,D
3,ISODME,D
4,No Kid Hungry,D
5,EyeWire,D
6,Barinsta,C
7,Dairy Queen,D
8,GNOME,C
9,Scrap.TF,D


In [88]:
#use the search function to find the identifier for the app to pass through the app() function
    #also, save any data from search() that's not in app() into new column

In [86]:
# Function to get the best match from `search` using fuzzy matching
def get_best_app_details(service_name):
    try:
        # Perform a search and get multiple results
        search_results = search(service_name, lang="en", country="us", n_hits=5)
        
        # Initialize variables for the best match
        best_match = None
        highest_score = 0
        
        # Iterate through the search results and use fuzzy matching on titles
        for result in search_results:
            title = result.get('title', '')
            match_score = fuzz.ratio(service_name.lower(), title.lower())
            
            # Check if this result is the best match based on the score threshold
            if match_score > highest_score:
                highest_score = match_score
                best_match = result
                
                # If match score is above a threshold (e.g., 90), stop early
                if match_score >= 99:
                    break
        
        # If a match was found, return its details
        if best_match:
            return pd.Series({
                'AppID': best_match.get('appId'),
                'Title': best_match.get('title'),
                'Genre': best_match.get('genre'),
                'Score': best_match.get('score'),
                'Installs': best_match.get('installs')
            })
        
        # If no close match is found, return empty values
        return pd.Series({
            'AppID': None,
            'Title': None,
            'Genre': None,
            'Score': None,
            'Installs': None
        })
    except Exception as e:
        print(f"Error searching for {service_name}: {e}")
        return pd.Series({
            'AppID': None,
            'Title': None,
            'Genre': None,
            'Score': None,
            'Installs': None
        })

# Apply the function to each row in 'Service' and expand to multiple columns
df_try2 = df.join(df['Service'].apply(get_best_app_details))

df_try2

,Service,Rating,AppID,Title,Summary,Genre,Score,Installs
0,AliExpress,D,com.alibaba.aliexpresshd,AliExpress - Shopping App,None,Shopping,4.490700,"500,000,000+"
1,MySudo,B,com.anonyome.mysudo,MySudo – Protect your identity,None,Communication,4.089888,"100,000+"
2,Rec Room,D,com.AgainstGravity.RecRoom,Rec Room - Play with friends!,None,Adventure,4.476791,"10,000,000+"
3,ISODME,D,com.episodeinteractive.android.catalog,Episode - Choose Your Story,None,Simulation,4.274074,"100,000,000+"
4,No Kid Hungry,D,com.playdatedigital.hungryhungryhippos,Hungry Hungry Hippos,None,Board,4.800000,100+
5,EyeWire,D,com.switch_android,Neurocycle,None,Health & Fitness,4.070000,"100,000+"
6,Barinsta,C,com.instagram.android,Instagram,None,Social,4.002759,"5,000,000,000+"
7,Dairy Queen,D,com.olo.dairyqueen.production,Dairy Queen® Food & Treats,None,Food & Drink,4.516776,"5,000,000+"
8,GNOME,C,com.FunFlavorGames.GnomesInc,Gnomes Inc.,None,Casual,NaN,"10,000+"
9,Scrap.TF,D,com.nicolaigd.scrapdivers,Scrap Divers,None,Action,4.767442,"10,000+"


In [103]:
# Function to clean and normalize strings by removing punctuation, extra spaces, and converting to lowercase
def clean_string(s):
    return re.sub(r'\s+', ' ', re.sub(r'[^\w\s]', '', s)).strip().lower()

# Function to check if Service is a subset of Title
def is_subset_match(service, title):
    if pd.isna(service) or pd.isna(title):
        return False  # Return False if either value is NaN
    clean_service = clean_string(service)
    clean_title = clean_string(title)
    return clean_service in clean_title

# Apply the function to create a new column 'CloseMatch'
df_try2['CloseMatch'] = df_try2.apply(lambda row: is_subset_match(row['Service'], row['Title']), axis=1)

# Separate into two DataFrames based on the 'CloseMatch' column
df_close_matches = df_try2[df_try2['CloseMatch']].drop(columns=['CloseMatch'])


In [109]:
df_close_matches

,Service,Rating,AppID,Title,Summary,Genre,Score,Installs
0,AliExpress,D,com.alibaba.aliexpresshd,AliExpress - Shopping App,None,Shopping,4.490700,"500,000,000+"
1,MySudo,B,com.anonyome.mysudo,MySudo – Protect your identity,None,Communication,4.089888,"100,000+"
2,Rec Room,D,com.AgainstGravity.RecRoom,Rec Room - Play with friends!,None,Adventure,4.476791,"10,000,000+"
7,Dairy Queen,D,com.olo.dairyqueen.production,Dairy Queen® Food & Treats,None,Food & Drink,4.516776,"5,000,000+"
8,GNOME,C,com.FunFlavorGames.GnomesInc,Gnomes Inc.,None,Casual,NaN,"10,000+"
11,Goodreads,E,com.goodreads,Goodreads - Find & Track Books,None,Books & Reference,3.072492,"10,000,000+"
12,NerdWallet,D,com.mobilecreditcards,NerdWallet: Manage Your Money,None,Finance,4.491146,"1,000,000+"
13,Aurora Store,D,org.shopage.demo,Aurora Store,None,Shopping,2.333333,"50,000+"
14,Too Good To Go,D,com.app.tgtg,Too Good To Go: End Food Waste,None,Food & Drink,4.815384,"50,000,000+"
18,Redbubble,C,com.redbubble,Redbubble,None,Shopping,3.353731,"1,000,000+"


In [117]:
#for each row in df_close_matches, use the AppID to get info that's useful but we don't yet have

# Function to get additional app details that are not already in the DataFrame
def get_additional_app_details(app_id):
    try:
        # Retrieve detailed information using the app id
        details = app(app_id, lang="en", country="us")
        return pd.Series({
            'Description': details.get('description'),
            'Price': details.get('price'),
            'Developer': details.get('developer'),
            'Content Rating': details.get('contentRating'),
            'App URL': details.get('url'),
            'Updated Date': details.get('updated'),
            'Ratings': details.get('ratings'),
            'Reviews': details.get('reviews'),
            'Free': details.get('free'),
            'Contains Ads': details.get('containsAds')
        })
    except Exception as e:
        print(f"Error retrieving details for app ID {app_id}: {e}")
        return pd.Series({
            'Description': None,
            'Price': None,
            'Developer': None,
            'Content Rating': None,
            'App URL': None,
            'Updated Date': None,
            'Ratings': None,
            'Reviews': None,
            'Free': None,
            'Contains Ads': None
        })

# Apply the function to each AppID in df_close_matches and expand to multiple columns
df_detailed_close_matches = df_close_matches.join(df_close_matches['AppID'].apply(get_additional_app_details))

df_detailed_close_matches

Service Rating                          AppID  \
0       AliExpress      D       com.alibaba.aliexpresshd   
1           MySudo      B            com.anonyome.mysudo   
2         Rec Room      D     com.AgainstGravity.RecRoom   
7      Dairy Queen      D  com.olo.dairyqueen.production   
8            GNOME      C   com.FunFlavorGames.GnomesInc   
11       Goodreads      E                  com.goodreads   
12      NerdWallet      D          com.mobilecreditcards   
13    Aurora Store      D               org.shopage.demo   
14  Too Good To Go      D                   com.app.tgtg   
18       Redbubble      C                  com.redbubble   

                             Title Summary              Genre     Score  \
0        AliExpress - Shopping App    None           Shopping  4.490700   
1   MySudo – Protect your identity    None      Communication  4.089888   
2    Rec Room - Play with friends!    None          Adventure  4.476791   
7       Dairy Queen® Food & Treats    None       Food & Drink  4.516776   
8                      Gnomes Inc.    None             Casual       NaN   
11  Goodreads - Find & Track Books    None  Books & Reference  3.072492   
12   NerdWallet: Manage Your Money    None            Finance  4.491146   
13                    Aurora Store    None           Shopping  2.333333   
14  Too Good To Go: End Food Waste    None       Food & Drink  4.815384   
18                       Redbubble    None           Shopping  3.353731   

        Installs  \
0   500,000,000+   
1       100,000+   
2    10,000,000+   
7     5,000,000+   
8        10,000+   
11   10,000,000+   
12    1,000,000+   
13       50,000+   
14   50,000,000+   
18    1,000,000+   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [119]:
#try to supplement with a different dataset for the bad ones!! Then if that works, you can merge it back into the good dataset

df_bad_matches

#FIGURE OUT A WAY TO TAKE IN DF_BAD_MATCHES AND DO THIS
    #ok, so i found a way through the "web shrinker api" but you have to pay for it so idk if it's worth it. get group's opinions
    #also could potentially use "SimilarWeb API". that one has a limited free version

,Service,Rating,AppID,Title,Summary,Genre,Score,Installs
3,ISODME,D,com.episodeinteractive.android.catalog,Episode - Choose Your Story,None,Simulation,4.274074,"100,000,000+"
4,No Kid Hungry,D,com.playdatedigital.hungryhungryhippos,Hungry Hungry Hippos,None,Board,4.800000,100+
5,EyeWire,D,com.switch_android,Neurocycle,None,Health & Fitness,4.070000,"100,000+"
6,Barinsta,C,com.instagram.android,Instagram,None,Social,4.002759,"5,000,000,000+"
9,Scrap.TF,D,com.nicolaigd.scrapdivers,Scrap Divers,None,Action,4.767442,"10,000+"
10,libera.chat,C,co.aureolin.coreirc,CoreIRC,None,Communication,4.027027,"1,000+"
15,Read Aloud,C,com.google.android.accessibility.reader,Reading mode,None,Tools,3.653465,"1,000,000+"
16,getrentback,A,com.yardi.systems.rentcafe.resident.livinglondon,Get Living Resident,None,Lifestyle,5.000000,"1,000+"
17,Open Humans,B,org.nypl.labs.OpenEbooks.app,Open eBooks,None,Books & Reference,3.020000,"100,000+"
19,Gap Creek Media,B,com.subsplashconsulting.s_4BJG6P,Coal Creek Media,None,Lifestyle,NaN,10+


In [ ]:
#probably gonna ignore df_bad_matches. so set this up to do input and output through gcp
    #then, run on the whole dataset
    #make sure to check first for thigns you used for testing - eg no longer need to drops rows where there was no rating, just do for all